In [13]:
import pandas as pd

run = pd.read_pickle("evaluations/testdev_base_clipbase_owllarge/results_snapshot.pkl")
run_with_skips = run
run = run[~run["skipped"]]
run["model_response"] = run["model_response"].apply(lambda x: x[0] if x != "UNSAT" else "UNSAT")

/var/folders/zf/q48kqz_93lz349jpplht237w0000gn/T/ipykernel_83005/664880102.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run["model_response"] = run["model_response"].apply(lambda x: x[0] if x != "UNSAT" else "UNSAT")


In [2]:
import re 

for i, row in run.iterrows():
    with open(f"../data/encoded_questions/{row['question_id']}.lp") as f:
        question_encoding = f.read()
    
    run.loc[i, "has_select"] = bool(re.search("select\(", question_encoding))
    run.loc[i, "has_filter"] = bool(re.search("filter\(", question_encoding))
    run.loc[i, "has_filter_any"] = bool(re.search("filter_any\(", question_encoding))
    run.loc[i, "has_relate"] = bool(re.search("relate\(", question_encoding)) 
    run.loc[i, "has_relate_any"] = bool(re.search("relate_any\(", question_encoding))
    run.loc[i, "has_relate_attr"] = bool(re.search("relate_attr\(", question_encoding))
    run.loc[i, "has_compare"] = bool(re.search("compare\(", question_encoding))
    run.loc[i, "has_query_attr"] = bool(re.search("query\(\w+, \w+, name\)", question_encoding))
    run.loc[i, "has_query_name"] = bool(re.search("query\(\w+, \w+, (?!name)\w+\)", question_encoding))
    run.loc[i, "has_verify_attr"] = bool(re.search("verify_attr\(", question_encoding))
    run.loc[i, "has_verify_rel"] = bool(re.search("verify_rel\(", question_encoding))
    run.loc[i, "has_choose_attr"] = bool(re.search("choose_attr\(", question_encoding))
    run.loc[i, "has_choose_rel"] = bool(re.search("choose_rel\(", question_encoding))
    
    run.loc[i, "has_exist"] = bool(re.search("exist\(", question_encoding))
    run.loc[i, "has_two_same"] = bool(re.search("two_same\(", question_encoding))
    run.loc[i, "has_two_different"] = bool(re.search("two_different\(", question_encoding))
    run.loc[i, "has_all_same"] = bool(re.search("all_same\(", question_encoding))
    run.loc[i, "has_all_different"] = bool(re.search("all_different\(", question_encoding))
    run.loc[i, "has_common"] = bool(re.search("common\(", question_encoding))
    run.loc[i, "has_and"] = bool(re.search("and\(", question_encoding))
    run.loc[i, "has_or"] = bool(re.search("or\(", question_encoding))

/tmp/ipykernel_1132731/3510796415.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run.loc[i, "has_select"] = bool(re.search("select\(", question_encoding))
/tmp/ipykernel_1132731/3510796415.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run.loc[i, "has_filter"] = bool(re.search("filter\(", question_encoding))
/tmp/ipykernel_1132731/3510796415.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [3]:
op_types = [t[4:] for t in list(run) if t.startswith("has_")]
percentages = []

# question type
percentages.append({
    "name": "all",
    "correct": run[run["correct"]].shape[0] / run.shape[0]
})
binary = run[(run["answer"] == "no") | (run["answer"] == "yes")]
percentages.append({
    "name": "binary",
    "correct": binary[binary["correct"]].shape[0] / binary.shape[0]
})
open = run[(run["answer"] != "no") & (run["answer"] != "yes")]
percentages.append({
    "name": "open",
    "correct": open[open["correct"]].shape[0] / open.shape[0]
})

# operations
for op in op_types:
    percentages.append({
        "name": op,
        "correct": run[run[f"has_{op}"] & run["correct"]].shape[0] / max(run[run[f"has_{op}"]].shape[0], 1)
    })
    
results = pd.DataFrame(percentages)
results["correct"] = results["correct"] * 100
print(results.style.format(precision=2).hide(axis="index").to_latex())


\begin{tabular}{lr}
name & correct \\
all & 41.46 \\
binary & 57.76 \\
open & 32.23 \\
select & 41.46 \\
filter & 43.71 \\
filter_any & 40.22 \\
relate & 38.58 \\
relate_any & 7.61 \\
relate_attr & 24.00 \\
compare & 30.43 \\
query_attr & 23.61 \\
query_name & 38.51 \\
verify_attr & 61.46 \\
verify_rel & 55.66 \\
choose_attr & 65.70 \\
choose_rel & 47.87 \\
exist & 58.01 \\
two_same & 42.48 \\
two_different & 40.58 \\
all_same & 80.95 \\
all_different & 60.00 \\
common & 14.58 \\
and & 56.46 \\
or & 56.03 \\
\end{tabular}



In [4]:
run[run["has_relate_any"]][["question_id", "semantic_str", "answer", "model_response"]]

,question_id,semantic_str,answer,model_response
6,20567512,"select: airplane (11)->relate: _,flying above,...",ocean,[moon]
9,201079951,select: window (0)->filter: open [0]->relate: ...,drapes,[mirror]
10,201079952,"select: window (0)->relate: _,around,s (12) [0...",drapes,[mirror]
13,202218649,"select: chalkboard (0)->relate: _,hanging abov...",picture,[toaster_oven]
24,20982539,select: person (6)->filter fatness: skinny [0]...,table,[sweater]
...,...,...,...,...
12537,201935187,"select: building (11)->relate: lamp,in front o...",tree,[dining_table]
12557,201935363,"select: camera (9)->relate: woman,to the right...",bricks,[dining_table]
12559,202040263,"select: platform (4)->relate: train,near,s (0)...",train station,[handbag]
12566,201535906,"select: rope (7)->relate: _,attached to,o (10)...",coat,UNSAT


In [25]:
run[run["correct"]].shape[0] / run.shape[0]

0.40289121202221495

In [5]:
run["runtime_sec"].mean()

0.9938848002138715

In [10]:
run["model_response"] = run["model_response"].str[0]

/tmp/ipykernel_466951/2138938923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run["model_response"] = run["model_response"].str[0]


In [5]:
run.to_csv("testdev_base_clipbase_owllarge.csv")

## Semantic Answer Similarity

In [77]:
import sys
import os
sys.path.insert(0, os.path.abspath("../src"))
sys.path.insert(0, os.path.abspath("../externals"))

from pipeline.encoding import sanitize_asp

def sanitize_answer(row):
    model_response = row["model_response"]
    answer = row["answer"]
    if model_response == "to_the_left_of" and answer == "left":
        return "to_the_left_of"
    elif model_response == "to_the_right_of" and answer == "right":
        return "to_the_right_of"
    elif model_response == "in_front_of" and answer == "front":
        return "in_front_of"
    else:
        return sanitize_asp(answer)

run["answer_sanitized"] = run.apply(sanitize_answer, axis=1)

/var/folders/zf/q48kqz_93lz349jpplht237w0000gn/T/ipykernel_83005/1484579369.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run["answer_sanitized"] = run.apply(sanitize_answer, axis=1)


In [80]:
run_open = run[(run["answer"] != "yes") & (run["answer"] != "no")]

In [81]:
from sentence_transformers import SentenceTransformer, util
st_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [82]:
def semantic_similarity(row):
    if row["model_response"] == "UNSAT":
        return 0.0
    else:
        encodings = st_model.encode([row["answer_sanitized"], row["model_response"]], convert_to_tensor=True)
        return util.cos_sim(encodings[0], encodings[1])[0,0].item()

run_open["answer_similarity"] = run_open.apply(semantic_similarity, axis=1)

/var/folders/zf/q48kqz_93lz349jpplht237w0000gn/T/ipykernel_83005/2207682893.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_open["answer_similarity"] = run_open.apply(semantic_similarity, axis=1)


In [83]:
run_open.to_csv("testdev_base_clipbase_owllarge_open.csv")

In [88]:
from nltk.corpus import wordnet as wn

def get_antonyms(word):
    antonyms = set()
    for syn in wn.synsets(word):
        for l in syn.lemmas():
            if l.antonyms():
                antonyms.add(l.antonyms()[0].name())
    return antonyms

run["is_antonym"] = run.apply(lambda row: row["model_response"].replace("_", " ") in get_antonyms(row["answer_sanitized"].replace("_", " ")), axis=1)

/var/folders/zf/q48kqz_93lz349jpplht237w0000gn/T/ipykernel_83005/3394329750.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run["is_antonym"] = run.apply(lambda row: row["model_response"].replace("_", " ") in get_antonyms(row["answer_sanitized"].replace("_", " ")), axis=1)


In [151]:
run_open[(run_open["answer_similarity"] > 0.60) & (run_open["answer_similarity"] < 0.99) & ~(run_open["is_antonym"])][["answer_sanitized", "model_response"]]

,answer_sanitized,model_response
50,white,blue
77,stove,gas_stove
84,light_brown,dark_brown
86,gown,dress
90,gown,dress
...,...,...
12527,sofa,couch
12528,sofa,couch
12531,sofa,couch
12550,black,gray


In [156]:
similar_answers = list(run_open[(run_open["answer_similarity"] > 0.50) & (run_open["answer_similarity"] < 0.99) & ~(run_open["is_antonym"])][["answer_sanitized", "model_response"]].iterrows())

In [157]:
similar_answers

[(22,
  answer_sanitized     white
  model_response      orange
  Name: 22, dtype: object),
 (38,
  answer_sanitized    concrete
  model_response      aluminum
  Name: 38, dtype: object),
 (40,
  answer_sanitized    concrete
  model_response      aluminum
  Name: 40, dtype: object),
 (50,
  answer_sanitized    white
  model_response       blue
  Name: 50, dtype: object),
 (64,
  answer_sanitized     dense
  model_response      sparse
  Name: 64, dtype: object),
 (71,
  answer_sanitized            beige
  model_response      cream_colored
  Name: 71, dtype: object),
 (77,
  answer_sanitized        stove
  model_response      gas_stove
  Name: 77, dtype: object),
 (81,
  answer_sanitized      field
  model_response      stadium
  Name: 81, dtype: object),
 (84,
  answer_sanitized    light_brown
  model_response       dark_brown
  Name: 84, dtype: object),
 (86,
  answer_sanitized     gown
  model_response      dress
  Name: 86, dtype: object),
 (88,
  answer_sanitized    glove
  model_re

In [134]:
len(similar_answers)

602

In [87]:
run_open[run_open["answer_similarity"] < 0.05].shape

(294, 24)